# Building an Intelligent Apartment Search Agent with Hyperbrowser

In this cookbook, we'll create a sophisticated apartment search system that combines autonomous web navigation with intelligent ranking capabilities. Our system will:

1. Search for apartments based on multiple criteria
2. Extract detailed property information
3. Intelligently rank results based on user preferences

This approach combines:
- **[Hyperbrowser](https://hyperbrowser.ai)** for autonomous web navigation
- **OpenAI's GPT-4** for intelligent result ranking

## Prerequisites

Before starting, you'll need:

1. A Hyperbrowser API key (sign up at [hyperbrowser.ai](https://hyperbrowser.ai) if you don't have one)
2. An OpenAI API key with access to GPT-4

Store these API keys in a `.env` file in the same directory as this notebook:

```
HYPERBROWSER_API_KEY=your_hyperbrowser_key_here
OPENAI_API_KEY=your_openai_key_here
```

## Step 1: Set Up Environment

In [ ]:
import os

from dotenv import load_dotenv
from hyperbrowser import AsyncHyperbrowser
from hyperbrowser.models import StartBrowserUseTaskParams
from hyperbrowser.models.session import CreateSessionParams
from openai import AsyncOpenAI

from IPython.display import Markdown, display
from typing import Optional

load_dotenv()

## Step 2: Initialize API Clients

We create instances of our API clients for web automation and natural language processing.

In [9]:
hb = AsyncHyperbrowser(api_key=os.getenv("HYPERBROWSER_API_KEY"))
llm = AsyncOpenAI()

## Step 3: Implement Apartment Search

Our search function uses Hyperbrowser's autonomous agent to navigate apartments.com. Key features:
- Dynamic filter construction from arguments
- Proxy support
- Structured markdown output

In [12]:
async def find_suitable_places_to_rent(user_params:str):
    resp = await hb.beta.agents.browser_use.start_and_wait(
        StartBrowserUseTaskParams(
            task=f"Go to apartments.com and find me a few places to rent. I have a few filters: {user_params}\n.Respond in markdown format.",
            session_options=CreateSessionParams(
                use_proxy=True
            ),
        )
    )
    if resp.data is not None:
        return resp.data.final_result
    return None

## Step 4: Search for Properties

We execute our search with specific criteria for San Francisco properties:

In [13]:
response = await find_suitable_places_to_rent(
    "I want the price range to be between 1000 and 5000. There should be at least 2 bedrooms and 1 bathroom. Search for apartments in San Francisco. Give me at least 5 results."
)
if response is not None:
    display(Markdown(response))
else:
    print("No response from the agent")

I have found 5 apartments in San Francisco with the specified filters:

1.  100 Van Ness, 100 Van Ness Ave, San Francisco, CA 94102, $4,748 - $5,088, 2 Beds, Pets Allowed, Fitness Center, Pool, Dishwasher, Refrigerator, Kitchen
2.  Parkmerced, 3711 19th Ave, San Francisco, CA 94132, $3,200 - $4,418, 2-3 Beds, Pets Allowed, Fitness Center, Clubhouse, Maintenance on site, Business Center, Elevator
3.  HQ, 1532 Harrison St, San Francisco, CA 94103, $3,501 - $3,643, 2 Beds, Pets Allowed, Fitness Center, Dishwasher, Kitchen, In Unit Washer & Dryer, Balcony, Range
4.  Hanover Soma West, 1140 Harrison St, San Francisco, CA 94103, $3,013 - $4,790, 2 Beds, Fitness Center, Dishwasher, Refrigerator, In Unit Washer & Dryer, Walk-In Closets, Clubhouse
5.  150 Van Ness, 150 Van Ness Ave, San Francisco, CA 94102, $3,635 - $5,800, 2 Beds, Pets Allowed, Fitness Center, Pool, Dishwasher, Refrigerator, Kitchen, In Unit Washer & Dryer

## Step 5: Implement Intelligent Ranking

We add a ranking function that uses GPT-4 to analyze and sort properties based on user preferences. This function:
- Takes listings and ranking criteria as input
- Uses a specialized system prompt for consistent analysis
- Returns ranked results with explanations

In [16]:
async def rank_apartments(markdown_listings: str, ranking_prompt: str) -> Optional[str]:
    system_prompt = """You are a helpful assistant that ranks apartment listings based on specific criteria.
    You will receive apartment listings in markdown format and a prompt with ranking criteria.
    Analyze the listings and return them in markdown format, ordered from best to worst match based on the criteria.
    Include a brief explanation for each ranking."""

    response = await llm.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": f"Here are the apartment listings:\n\n{markdown_listings}\n\nRank these apartments based on the following criteria:\n{ranking_prompt}"}
        ]
    )
    
    return response.choices[0].message.content

## Step 6: Apply Intelligent Ranking

We apply our ranking system to the search results, prioritizing pet-friendly and affordable properties:

In [17]:
if response is not None:
    ranked_response = await rank_apartments(response, "I want an apartment that allows me to have a pet, and preferably is on the cheaper side.")
    display(Markdown(ranked_response))
else:
    print("No response from the agent")

Here are the apartment listings ranked based on your criteria:

1.  Parkmerced, 3711 19th Ave, San Francisco, CA 94132, $3,200 - $4,418, 2-3 Beds, Pets Allowed, Fitness Center, Clubhouse, Maintenance on site, Business Center, Elevator
    - This option is the cheapest among the pet-friendly apartments and therefore the best match for your criteria.

2.  HQ, 1532 Harrison St, San Francisco, CA 94103, $3,501 - $3,643, 2 Beds, Pets Allowed, Fitness Center, Dishwasher, Kitchen, In Unit Washer & Dryer, Balcony, Range
    - Although more expensive than Parkmerced, this complex is still less expensive than the other options and it also allows pets.

3.  150 Van Ness, 150 Van Ness Ave, San Francisco, CA 94102, $3,635 - $5,800, 2 Beds, Pets Allowed, Fitness Center, Pool, Dishwasher, Refrigerator, Kitchen, In Unit Washer & Dryer
    - This apartment allows pets but its price ranges up to $5,800 which is higher compared to the first two options.

4.  100 Van Ness, 100 Van Ness Ave, San Francisco, CA 94102, $4,748 - $5,088, 2 Beds, Pets Allowed, Fitness Center, Pool, Dishwasher, Refrigerator, Kitchen
    - Despite it being a pet-friendly environment, the apartment's minimum price is significantly higher than the others.

5.  Hanover Soma West, 1140 Harrison St, San Francisco, CA 94103, $3,013 - $4,790, 2 Beds, Fitness Center, Dishwasher, Refrigerator, In Unit Washer & Dryer, Walk-In Closets, Clubhouse
    - Even though this option could be within your budget, it does not allow pets which makes it the least suitable for your needs.

## Advancing from Basic to Advanced Autonomous Agents

This apartment search example represents a significant advancement over the basic Hacker News summarizer we built in the previous notebook. Both examples utilize autonomous agents, but this implementation demonstrates several key evolutionary steps:

**From Simple to Parameterized Tasks**  
Besides the basic prompt, which dictated the only behaviour that our agent could do, we now accept parameters in a more natural language. This enables a more flexible and reusable agent

**From Navigation to Complex Interaction**  
The apartment search agent engages in more sophisticated web interactions, including form filling, filtering results, and extracting structured data.

**Multi-Stage Pipelines**  
We now have a multi-stage approach where the autonomous web agent is followed by a specialized analysis component. The initial web navigation and data extraction is handled by the autonomous agent, while the subsequent operations are performed by a separate component. This enables each component to focus on its strengths.

These advancements illustrate how the fundamental concept of autonomous agents can be extended and enhanced for more sophisticated applications. By combining autonomous web navigation with intelligent analysis, parameterized inputs, and advanced session handling, we can create powerful solutions for complex real-world problems while maintaining the simplicity and adaptability that make autonomous agents so valuable.

## Conclusion

This example demonstrates the power of combining autonomous web agents with intelligent analysis. Our system:

1. Autonomously navigates complex real estate websites
2. Handles advanced features like CAPTCHAs and proxies
3. Extracts structured property information
4. Applies intelligent ranking based on natural language preferences

The result is a sophisticated apartment search tool that not only finds properties matching specific criteria but also intelligently ranks them according to user preferences. This combination of autonomous web navigation and intelligent analysis creates a more powerful and user-friendly solution than either capability alone could provide.

## Relevant Links
- [Hyperbrowser](https://hyperbrowser.ai)
- [OpenAI Docs](https://platform.openai.com/docs/introduction)
- [Hyperbrowser Browser-use cookbook]("/cookbook/browser-use")